In [1]:
import pandas as pd
import numpy as np
import os
import time
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras.regularizers import Regularizer
# Figure out how to import regularizers
import tensorflow.keras.utils as ku 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


In [4]:
#The dataset (P.S. Look, it is TSV)
data=pd.read_csv('happy.csv')

# removing null values to avoid errors 
data.dropna(inplace = True) 
  
# converting to string data type
data["Title"]= data["Title"].astype(str)
  
# slicing till 2nd last element
data["lyric"]= data["Title"].str.slice(10,)
  
# display
data.head()

# data.album.drop_duplicates().tolist()
data.head()

,Title,lyric
0,[00:26.30]The earth is shaking under siege,The earth is shaking under siege
1,[00:32.63]And every breath will meet its fate,And every breath will meet its fate
2,[00:38.25]Still we hunger for a moment of freedom,Still we hunger for a moment of freedom
3,[00:44.43]Even though the hour is late,Even though the hour is late
4,[00:47.17],


In [5]:
data.lyric.drop_duplicates().tolist()

['The earth is shaking under siege',
 'And every breath will meet its fate',
 'Still we hunger for a moment of freedom',
 'Even though the hour is late',
 '',
 'I find hope and it gives me rest',
 'I find hope in a beating chest',
 "I find hope in what eyes don\\'t see",
 'I find hope in your hate for me',
 'Have no fear when the waters rise',
 'We can conquer this great divide',
 'When every eye is on the fortune',
 'It can only breed contempt',
 'They say blood is thicker than oceans',
 'Still we box our brothers in',
 'Whoa, ooo, ooo, ooo',
 "And we\\'re gone",
 "And we\\'re holding on",
 'Have no fear',
 'landlords Knocking at my door cussing me out',
 'got laid off my job the night before',
 "can't figure how",
 "I'm gonna fix tomorrow away",
 "If today's still a mess",
 "Can u tell me what's the point man",
 'It all seems meaningless',
 'I wish that I could step away and breathe',
 "This world's trying to swallow me",
 'Clear away the clouds inside my head',
 'Someone just tell m

In [6]:
data.shape

(5241, 2)

In [7]:
IND = data.lyric.drop_duplicates().tolist()
#Move in the index range
CHORUSES=[]
for i in IND:
    CHORUSES.append(data.lyric.tolist())

In [8]:
CHORUSES=np.array(data.lyric.tolist())
CHORUSES=CHORUSES.tolist()
#Write everything in a single line 
TERM=''
for c in range(len(CHORUSES)):
    
    TERM= TERM+ CHORUSES[c]+ ' \n '

In [9]:
TERM

'The earth is shaking under siege \n And every breath will meet its fate \n Still we hunger for a moment of freedom \n Even though the hour is late \n  \n I find hope and it gives me rest \n I find hope in a beating chest \n I find hope in what eyes don\\\'t see \n I find hope in your hate for me \n Have no fear when the waters rise \n We can conquer this great divide \n  \n When every eye is on the fortune \n It can only breed contempt \n They say blood is thicker than oceans \n Still we box our brothers in \n  \n I find hope and it gives me rest \n I find hope in a beating chest \n I find hope in what eyes don\\\'t see \n I find hope in your hate for me \n Have no fear when the waters rise \n We can conquer this great divide \n  \n Whoa, ooo, ooo, ooo \n Whoa, ooo, ooo, ooo \n Whoa, ooo, ooo, ooo \n And we\\\'re gone \n And we\\\'re gone \n And we\\\'re holding on \n And we\\\'re holding on \n And we\\\'re holding on \n  \n I find hope and it gives me rest \n I find hope in a beating

In [10]:
print("The unique characters:",sorted(set(TERM)))

The unique characters: ['\t', '\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '‘', '’', '“', '”', '′', '。', '＇', '（', '）']


In [11]:
to_remove = ['[', '\\', ']', '_', '{', '}','~', '§', '…','≡', '了', '人', '作', '制', '卐', '我', '朝', '王', '语', '\ufeff','(', ')','*', '（', '）', '‘', '’', '′', '＇', "'", "'", '.']
for symbol in to_remove:
    TERM = TERM.replace(symbol,"")

print("The unique characters:",sorted(set(TERM)))

The unique characters: ['\t', '\n', ' ', '!', '"', '&', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '“', '”', '。']


In [12]:
#Write it into a file
f = open("happychorus.txt", "a")
f.write(TERM)
f.close()

In [13]:
text = open('happychorus.txt', 'rb').read().decode(encoding='utf-8')

In [14]:
text[0:5000]

'The earth is shaking under siege \n And every breath will meet its fate \n Still we hunger for a moment of freedom \n Even though the hour is late \n  \n I find hope and it gives me rest \n I find hope in a beating chest \n I find hope in what eyes dont see \n I find hope in your hate for me \n Have no fear when the waters rise \n We can conquer this great divide \n  \n When every eye is on the fortune \n It can only breed contempt \n They say blood is thicker than oceans \n Still we box our brothers in \n  \n I find hope and it gives me rest \n I find hope in a beating chest \n I find hope in what eyes dont see \n I find hope in your hate for me \n Have no fear when the waters rise \n We can conquer this great divide \n  \n Whoa, ooo, ooo, ooo \n Whoa, ooo, ooo, ooo \n Whoa, ooo, ooo, ooo \n And were gone \n And were gone \n And were holding on \n And were holding on \n And were holding on \n  \n I find hope and it gives me rest \n I find hope in a beating chest \n I find hope in wha

In [15]:
# Splitting the string into sentences, while converting whole data into lowercase.
corpus = text.lower().split("\n")
# To make sure no sentence appears twice in our corpus, we use set. Otherwise, it will make the model biased.
corpus = list(set(corpus))
print(corpus)

[' look straight ahead, thats nothing but blue skies ', ' your lipstick stains ', ' and i hope i can see ', ' its gonna be me ', ' so with enough that i can afford ', ' that baby, im relieved ', ' everyone is against me and the world wants to fight me ', ' i dont wanna say im leaving ', ' cause when im here, im always happy ', ' so i cry, and i pray for you too ', ' i fell apart and combed my hair ', ' were gon keep on dancing, out on the floor, ', ' we cut high but were cutting out anyway ', ' here is the rainbow ive been prayin for ', ' its hard to say id rather stay ', ' have no fear ', ' lucky to have been where i have been ', ' tle darling, i feel that ice is slowly melting ', ' baby can you feel me ', ' life, thats city life, yeah thats city life, thats city life ', ' open up your mind and see like me i wont hesitate ', ' because its cloudy out there ', ' for something i wouldnt dare miss ', ' dementia youre driving me crazy ', ' i hate to break it to you babe ', ' i could be you

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [17]:
tokenizer.word_index

{'i': 1,
 'the': 2,
 'you': 3,
 'and': 4,
 'a': 5,
 'to': 6,
 'me': 7,
 'my': 8,
 'your': 9,
 'in': 10,
 'im': 11,
 'it': 12,
 'that': 13,
 'all': 14,
 'on': 15,
 'love': 16,
 'of': 17,
 'be': 18,
 'its': 19,
 'dont': 20,
 'is': 21,
 'we': 22,
 'oh': 23,
 'so': 24,
 'for': 25,
 'but': 26,
 'as': 27,
 'do': 28,
 'like': 29,
 'know': 30,
 'just': 31,
 'youre': 32,
 'got': 33,
 'what': 34,
 'can': 35,
 'when': 36,
 'no': 37,
 'now': 38,
 'if': 39,
 'out': 40,
 'down': 41,
 'with': 42,
 'up': 43,
 'get': 44,
 'are': 45,
 'this': 46,
 'cause': 47,
 'have': 48,
 'go': 49,
 'were': 50,
 'say': 51,
 'way': 52,
 'time': 53,
 'feel': 54,
 'well': 55,
 'day': 56,
 'one': 57,
 'gonna': 58,
 'could': 59,
 'never': 60,
 'make': 61,
 'wanna': 62,
 'from': 63,
 'come': 64,
 'at': 65,
 'see': 66,
 'not': 67,
 'baby': 68,
 'was': 69,
 'right': 70,
 'ive': 71,
 'every': 72,
 'cant': 73,
 'will': 74,
 'back': 75,
 'keep': 76,
 'let': 77,
 'ill': 78,
 'night': 79,
 'want': 80,
 'she': 81,
 'here': 82,
 'ye

In [18]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [19]:
input_sequences

[[101, 431],
 [101, 431, 621],
 [101, 431, 621, 103],
 [101, 431, 621, 103, 142],
 [101, 431, 621, 103, 142, 26],
 [101, 431, 621, 103, 142, 26, 148],
 [101, 431, 621, 103, 142, 26, 148, 622],
 [9, 1181],
 [9, 1181, 803],
 [4, 1],
 [4, 1, 215],
 [4, 1, 215, 1],
 [4, 1, 215, 1, 35],
 [4, 1, 215, 1, 35, 66],
 [19, 58],
 [19, 58, 18],
 [19, 58, 18, 7],
 [24, 42],
 [24, 42, 276],
 [24, 42, 276, 13],
 [24, 42, 276, 13, 1],
 [24, 42, 276, 13, 1, 35],
 [24, 42, 276, 13, 1, 35, 517],
 [13, 68],
 [13, 68, 11],
 [13, 68, 11, 1182],
 [250, 21],
 [250, 21, 623],
 [250, 21, 623, 7],
 [250, 21, 623, 7, 4],
 [250, 21, 623, 7, 4, 2],
 [250, 21, 623, 7, 4, 2, 105],
 [250, 21, 623, 7, 4, 2, 105, 624],
 [250, 21, 623, 7, 4, 2, 105, 624, 6],
 [250, 21, 623, 7, 4, 2, 105, 624, 6, 306],
 [250, 21, 623, 7, 4, 2, 105, 624, 6, 306, 7],
 [1, 20],
 [1, 20, 62],
 [1, 20, 62, 51],
 [1, 20, 62, 51, 11],
 [1, 20, 62, 51, 11, 625],
 [47, 36],
 [47, 36, 11],
 [47, 36, 11, 82],
 [47, 36, 11, 82, 11],
 [47, 36, 11, 82, 

In [20]:
corpus[0]

' look straight ahead, thats nothing but blue skies '

In [21]:
tokenizer.word_index["still"]

168

In [22]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [23]:
input_sequences

array([[   0,    0,    0, ...,    0,  101,  431],
       [   0,    0,    0, ...,  101,  431,  621],
       [   0,    0,    0, ...,  431,  621,  103],
       ...,
       [   0,    0,    0, ...,   11, 2542,  210],
       [   0,    0,    0, ..., 2542,  210,    2],
       [   0,    0,    0, ...,  210,    2, 2543]], dtype=int32)

In [24]:
input_sequences.shape

(15596, 19)

In [25]:
tokenizer.word_index['universe']

809

In [26]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [27]:
label.shape

(15596, 2544)

In [28]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))  #(# Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences=True)))  #(# An LSTM Layer)
model.add(Dropout(0.2))  #(# A dropout layer)
model.add(LSTM(100))  #(# Another LSTM Layer)
model.add(Dense(total_words/2, activation='relu'))  #(# A Dense Layer including regularizers)
model.add(Dense(total_words, activation='softmax'))  #(# A Dense Layer)
# Pick an optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  #(# Pick a loss function and an optimizer)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           254400    
                                                                 
 bidirectional (Bidirectiona  (None, 18, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 18, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1272)              128472    
                                                                 
 dense_1 (Dense)             (None, 2544)              3238512   
                                                        

In [29]:
model.fit(predictors, label, epochs= 50, verbose=1)

Epoch 1/50
488/488 [==============================] - 14s 10ms/step - loss: 6.3308 - accuracy: 0.0392
Epoch 2/50
488/488 [==============================] - 5s 10ms/step - loss: 5.8612 - accuracy: 0.0473
Epoch 3/50
488/488 [==============================] - 5s 11ms/step - loss: 5.5798 - accuracy: 0.0565
Epoch 4/50
488/488 [==============================] - 5s 10ms/step - loss: 5.3899 - accuracy: 0.0687
Epoch 5/50
488/488 [==============================] - 5s 10ms/step - loss: 5.1917 - accuracy: 0.0825
Epoch 6/50
488/488 [==============================] - 7s 14ms/step - loss: 5.0136 - accuracy: 0.0948
Epoch 7/50
488/488 [==============================] - 5s 11ms/step - loss: 4.8378 - accuracy: 0.1052
Epoch 8/50
488/488 [==============================] - 5s 11ms/step - loss: 4.6769 - accuracy: 0.1198
Epoch 9/50
488/488 [==============================] - 5s 10ms/step - loss: 4.5118 - accuracy: 0.1283
Epoch 10/50
488/488 [==============================] - 5s 10ms/step - loss: 4.3557 - accur

In [30]:
from keras.models import load_model


In [33]:
model.save('happy_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

In [31]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict(token_list, verbose=0)
        classes = np.argmax(predicted,axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == classes:
                output_word = word
                break
        seed_text += " " + output_word
    print(str(seed_text))

yhat_classes = model.predict_classes(X_test)

predict_x=model.predict(X_test) 

classes_x=np.argmax(predict_x,axis=1)

model.predict(x_test) > 0.5).astype("int32")

In [ ]:
model = load_model('happy_model.h5')

In [32]:
make_lyrics("alive", 10)

alive july aint good to me pullin u back to be
